In [ ]:
# Need for converting country name
!pip install -q pycountry
!pip install -q pycountry-convert
!pip install -q jellyfish  #approximate & phonetic matching of strings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.2 MB/s eta 0:00:00


In [ ]:

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pycountry
import pycountry_convert as pc

from difflib import SequenceMatcher # used for comparing pairs of input sequences
import jellyfish

import re

TEMPLATE = 'simple_white'
WIDTH = 1200

INDIA_ORANGE = "rgb(255, 154, 47)"
INDIA_GREEN = "rgb(10, 137, 1)"
INDIA_BLUE = "rgb(0, 0, 137)"

COLOR_DICT = {
    'North America':INDIA_ORANGE,
    'Asia':INDIA_GREEN,
    'Oceania':INDIA_BLUE,
    'Europe':'GoldenRod',
    'Africa':'LightSeaGreen',
    'South America':'PaleVioletRed'
}

In [ ]:
india_imports_df = pd.read_csv("/content/India_exports_FY22.csv", thousands=',')
india_exports_df = pd.read_csv("/content/India_imports_FY22.csv", thousands=',')

### **DATA PREPROCESSING**

In [ ]:
print("##India imports data##")
display(india_imports_df.head(2))
india_imports_df.info()

print("\n##India exports data##")
display(india_exports_df.head(2))
india_exports_df.info()


##India imports data##


,S.No.,Country,2020-2021,%Share,2021-2022,%Share1,%Growth
0,1,AFGHANISTAN,825.78,0.283,554.47,0.1314,-32.85
1,2,ALBANIA,46.75,0.016,55.16,0.0131,17.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   S.No.       235 non-null    int64  
 1   Country     235 non-null    object 
 2   2020-2021   232 non-null    float64
 3   %Share      235 non-null    object 
 4   2021-2022   230 non-null    float64
 5   %Share1     235 non-null    object 
 6   %Growth     235 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 13.0+ KB

##India exports data##


,S.No.,Country,2020-2021,%Share,2021-2022,%Share1,%Growth
0,1,AFGHANISTAN,509.49,0.1292,510.93,0.0834,0.28
1,2,ALBANIA,8.30,0.0021,30.71,0.005,269.78


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   S.No.       227 non-null    int64  
 1   Country     227 non-null    object 
 2   2020-2021   225 non-null    float64
 3   %Share      227 non-null    object 
 4   2021-2022   216 non-null    float64
 5   %Share1     227 non-null    object 
 6   %Growth     227 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 12.5+ KB


In [ ]:
def prep_df(df):
    num_cols = ['2020-2021', '%Share', '2021-2022', '%Share1', '%Growth']

    df.columns = [c.strip() for c in df.columns] # remove space
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce') # converted to numeric types
    df = df.fillna(0) # fillna
    return df

india_imports_df = prep_df(india_imports_df)
india_exports_df = prep_df(india_exports_df)
print("##India imports data##")
display(india_imports_df.head(10))
india_imports_df.info()

print("##India exports data##")
display(india_exports_df.head(10))
india_exports_df.info()

##India imports data##


,S.No.,Country,2020-2021,%Share,2021-2022,%Share1,%Growth
0,1,AFGHANISTAN,825.78,0.2830,554.47,0.1314,-32.85
1,2,ALBANIA,46.75,0.0160,55.16,0.0131,17.99
2,3,ALGERIA,594.74,0.2038,703.25,0.1667,18.24
3,4,AMERI SAMOA,0.58,0.0002,0.78,0.0002,33.68
4,5,ANDORRA,0.02,0.0000,0.05,0.0000,141.87
5,6,ANGOLA,259.60,0.0890,452.45,0.1072,74.29
6,7,ANGUILLA,0.05,0.0000,0.09,0.0000,80.67
7,8,ANTARTICA,1.05,0.0004,0.07,0.0000,-92.98
8,9,ANTIGUA,1.62,0.0006,2.47,0.0006,53.17
9,10,ARGENTINA,687.84,0.2357,1425.94,0.3380,107.31


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S.No.      235 non-null    int64  
 1   Country    235 non-null    object 
 2   2020-2021  235 non-null    float64
 3   %Share     235 non-null    float64
 4   2021-2022  235 non-null    float64
 5   %Share1    235 non-null    float64
 6   %Growth    235 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 13.0+ KB
##India exports data##


,S.No.,Country,2020-2021,%Share,2021-2022,%Share1,%Growth
0,1,AFGHANISTAN,509.49,0.1292,510.93,0.0834,0.28
1,2,ALBANIA,8.30,0.0021,30.71,0.0050,269.78
2,3,ALGERIA,408.79,0.1036,1004.24,0.1639,145.66
3,4,AMERI SAMOA,0.96,0.0002,0.00,0.0000,-99.74
4,5,ANDORRA,0.01,0.0000,0.00,0.0000,0.00
5,6,ANGOLA,1879.74,0.4766,2725.08,0.4448,44.97
6,7,ANGUILLA,0.18,0.0000,0.03,0.0000,-83.84
7,8,ANTARTICA,0.67,0.0002,1.22,0.0002,82.37
8,9,ANTIGUA,0.14,0.0000,0.13,0.0000,-6.05
9,10,ARGENTINA,2627.05,0.6660,4201.74,0.6859,59.94


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S.No.      227 non-null    int64  
 1   Country    227 non-null    object 
 2   2020-2021  227 non-null    float64
 3   %Share     227 non-null    float64
 4   2021-2022  227 non-null    float64
 5   %Share1    227 non-null    float64
 6   %Growth    227 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 12.5+ KB


**MERGE AND ADD TOTAL**

In [ ]:
#Since each of the import and export data has the same format, we can merge the data based on the country. We can also create a new column.

# used "import" and "export" as suffixes to distinguish.
#Total is the sum of imports and exports.
#also calculated the share and growth rate for total.

# Merge
temp1 = india_imports_df.drop(columns='S.No.')
temp1.columns = ['Country'] + [f"{c}_import" for c in temp1.columns if c != 'Country']
temp2 = india_exports_df.drop(columns='S.No.')
temp2.columns = ['Country'] + [f"{c}_export" for c in temp2.columns if c != 'Country']
india_df = pd.merge(temp1, temp2, on='Country')

# Add Total
for i, c in enumerate(['2020-2021', '2021-2022']):
    india_df[f"{c}_total"] = india_df[f"{c}_import"] + india_df[f"{c}_export"]
    n = '' if i==0 else '1'
    india_df[f"%Share{n}_total"] = 100 * india_df[f"{c}_total"] / india_df[f"{c}_total"].sum()

india_df[f"%Growth_total"] = 100*(india_df["2021-2022_total"] - india_df["2020-2021_total"]) / india_df["2020-2021_total"]

india_df.head()

,Country,2020-2021_import,%Share_import,2021-2022_import,%Share1_import,%Growth_import,2020-2021_export,%Share_export,2021-2022_export,%Share1_export,%Growth_export,2020-2021_total,%Share_total,2021-2022_total,%Share1_total,%Growth_total
0,AFGHANISTAN,825.78,0.2830,554.47,0.1314,-32.85,509.49,0.1292,510.93,0.0834,0.28,1335.27,0.194577,1065.40,0.102987,-20.210894
1,ALBANIA,46.75,0.0160,55.16,0.0131,17.99,8.30,0.0021,30.71,0.0050,269.78,55.05,0.008022,85.87,0.008301,55.985468
2,ALGERIA,594.74,0.2038,703.25,0.1667,18.24,408.79,0.1036,1004.24,0.1639,145.66,1003.53,0.146235,1707.49,0.165055,70.148376
3,AMERI SAMOA,0.58,0.0002,0.78,0.0002,33.68,0.96,0.0002,0.00,0.0000,-99.74,1.54,0.000224,0.78,0.000075,-49.350649
4,ANDORRA,0.02,0.0000,0.05,0.0000,141.87,0.01,0.0000,0.00,0.0000,0.00,0.03,0.000004,0.05,0.000005,66.666667


**Country name**

In [ ]:
def convert_country_name(x):
    cname = x[:]

    try:
        # Search without all spaces in existing country names and add them to the candidate list.
        cands = [c.name for c in pycountry.countries.search_fuzzy(cname.replace(" ", ""))]
    except:
        cands = []

    try:
        # The abbreviations 'U' and 'RP' of the existing country names are changed to 'United' and 'RP', respectively.
        cname = re.sub('[^a-zA-z]', " ", cname)
        for st, tt in zip(['U', 'RP', 'IS', 'DP'], ['United', 'Republic', 'Islands', 'Democratic People']):
            cname = re.sub(f'{st}[\s]+',f"{tt} ", cname)
            cname = re.sub(f'[\s]+{st}', f" {tt}",cname)

        # Then, for each token, add a group of candidates using pycountry's fuzzy search.
        for sub in cname.split(" "):
            if len(sub)<=2:
                continue
            try:
                cands += [c.name for c in pycountry.countries.search_fuzzy(sub)]
            except:
                continue

        # Using the string similarity of jellyfish, find the most similar country name among the candidate groups.
        cand_sim = [jellyfish.jaro_distance(cname.lower(), c.lower()) for c in cands]
        return cands[np.argmax(cand_sim)]
    except:
        print(f"Error : {x}")
        return np.nan

In [ ]:

print("#### Sample Test ####")
for c in ['U ARAB EMTS ', 'U S A', 'CHINA P RP', 'KOREA RP', 'BOSNIA-HRZGOVIN', 'N. MARIANA IS.', 'BAHARAIN IS', 'ANTARTICA']:
    print(f"{c} ==> {convert_country_name(c)}")

#### Sample Test ####
U ARAB EMTS  ==> United Arab Emirates
U S A ==> United States
CHINA P RP ==> China
KOREA RP ==> Korea, Republic of
BOSNIA-HRZGOVIN ==> Bosnia and Herzegovina
N. MARIANA IS. ==> Northern Mariana Islands
BAHARAIN IS ==> Bahamas
Error : ANTARTICA
ANTARTICA ==> nan


In [ ]:
print("#### Convert All ####")
india_df['Name'] = india_df['Country'].apply(lambda x : convert_country_name(x))

error_country = {
    "ANTARTICA " : "Antarctica",
    "KYRGHYZSTAN " : "Kyrgyzstan",
    "NETHERLANDANTIL " : "Netherlands Antilles",
    "SWAZILAND " : "Eswatini",
    "UNSPECIFIED " : "UNSPECIFIED"
}

india_df['Name'] = india_df['Name'].fillna(india_df['Country'])
india_df['Name'] = india_df['Name'].apply(lambda x : error_country[x] if x in error_country else x)

#### Convert All ####
Error : ANTARTICA 
Error : KYRGHYZSTAN 
Error : NETHERLANDANTIL 
Error : SWAZILAND 
Error : UNSPECIFIED 


In [ ]:
def get_alph_3_name(x):
    try:
        return pycountry.countries.lookup(x).alpha_3
    except:
        return 'ZZZ' # No data name

india_df['Name_3'] = india_df['Name'].apply(lambda x : get_alph_3_name(x))
india_df[['Country', 'Name', 'Name_3']].head(10)

,Country,Name,Name_3
0,AFGHANISTAN,Afghanistan,AFG
1,ALBANIA,Albania,ALB
2,ALGERIA,Algeria,DZA
3,AMERI SAMOA,American Samoa,ASM
4,ANDORRA,Andorra,AND
5,ANGOLA,Angola,AGO
6,ANGUILLA,Anguilla,AIA
7,ANTARTICA,Antarctica,ATA
8,ANTIGUA,Antigua and Barbuda,ATG
9,ARGENTINA,Argentina,ARG


**Continental name**

In [ ]:
#we get a continental name based on a 3-letter abbreviation so that we can count continents. The name of the continent can be obtained by using pycountry_convert.
def get_continet_name(name_3):
    try:
        alpah2 = pc.country_alpha3_to_country_alpha2(name_3)
        continent_code = pc.country_alpha2_to_continent_code(alpah2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
    except:
        continent_name = 'Others'
    return continent_name

india_df['Continent'] = india_df['Name_3'].apply(lambda x : get_continet_name(x))
india_df[['Country', 'Name', 'Name_3', 'Continent']].head(3)

,Country,Name,Name_3,Continent
0,AFGHANISTAN,Afghanistan,AFG,Asia
1,ALBANIA,Albania,ALB,Europe
2,ALGERIA,Algeria,DZA,Africa


# **Visualization**

In [ ]:
#trade volume by continent
# Asia has a similar number of countries compared to Europe, but much more total trade.
# India trades with many countries in Africa, but the total volume of trade is not large.
target = '2021-2022_total'
show_df = india_df[india_df['Continent']!='Others'].groupby(['Continent'])[[target]].sum().sort_values(by=[target], ascending=False)
show_df['Cnt'] = india_df.groupby(['Continent'])[[target]].size()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=show_df.index, y=show_df[target], marker_color=INDIA_GREEN, name='Total'))
fig.add_trace(go.Scatter(x=show_df.index, y=show_df['Cnt'], marker_color=INDIA_ORANGE, name='Number of Country'), secondary_y=True)

fig.update_layout(title_text=f'<b>The largest trade volume by continent ({target})</b>', template=TEMPLATE)
fig.update_yaxes(title='Total')
fig.update_yaxes(title='Count', secondary_y=True)
fig.show()

In [ ]:
# Is there anything unusual compared to the previous year
temp_show_df = pd.DataFrame()
for cate in ['total', 'import', 'export']:
    c1 = f'2020-2021_{cate}'
    c2 = f'2021-2022_{cate}'
    temp_ = pd.DataFrame(india_df[[c1, c2]].sum()).T
    temp_[f'%Growth_{cate}'] = 100* (temp_[c2] - temp_[c1]) / temp_[c1]
    temp_.columns = ['2020-2021', '2021-2022', 'Growth_Rate(%)']
    temp_show_df = temp_show_df.append(temp_)

temp_show_df.index = ['total', 'import', 'export']
display(temp_show_df)

BASE_GR = {
    'total': 50.748297,
    'import': 55.312213,
    'export' : 44.579255
}

<ipython-input-28-89f533d27193>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-89f533d27193>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-89f533d27193>:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,2020-2021,2021-2022,Growth_Rate(%)
total,686243.34,1034500.15,50.748297
import,291807.41,421892.98,44.579255
export,394435.93,612607.17,55.312213


# **REGRESSION USING PIPELINE**

# **Data Preparation**

In [ ]:
y_col = '2021-2022_total'
num_cols = ['2020-2021_import', '%Share_import', '2020-2021_export', '%Share_export', '2020-2021_total', '%Share_total']
cat_cols = ['Continent']
x_cols = num_cols + cat_cols

print(f"Y : {y_col}")
print(f"X numeric : {num_cols}")
print(f"X category : {cat_cols}")

Y : 2021-2022_total
X numeric : ['2020-2021_import', '%Share_import', '2020-2021_export', '%Share_export', '2020-2021_total', '%Share_total']
X category : ['Continent']


In [ ]:
random_seed = 1
lost_rate = 0.3
sample_n = int(lost_rate * len(india_df))


df = india_df[['Country', y_col]+x_cols].copy()
test_name = df['Country'].sample(n=sample_n, random_state=random_seed).tolist()
test_df = df[df['Country'].isin(test_name)]
train_df = df[~df['Country'].isin(test_name)]

display(df.head(3))
print(f"Total data size : {len(df)}")
print(f"Test data size : {len(test_df)}")
print(f"Train data size : {len(train_df)}")

X_train = train_df.drop(columns=[y_col])
y_train = train_df[y_col]

X_test = test_df.drop(columns=[y_col])
y_test = test_df[y_col]

,Country,2021-2022_total,2020-2021_import,%Share_import,2020-2021_export,%Share_export,2020-2021_total,%Share_total,Continent
0,AFGHANISTAN,1065.40,825.78,0.2830,509.49,0.1292,1335.27,0.194577,Asia
1,ALBANIA,85.87,46.75,0.0160,8.30,0.0021,55.05,0.008022,Europe
2,ALGERIA,1707.49,594.74,0.2038,408.79,0.1036,1003.53,0.146235,Africa


Total data size : 225
Test data size : 67
Train data size : 158


# **Make Pipeline**

In [ ]:
#Configure the Pipeline using sklearn. The details of the pipeline are as follows.

#Use StandardScaler to convert the data in the numeric column.
#Use OneHotEncoder to convert the data in the category column.
#Use the KFold to perform cross-validation. (k=5)
#Use MAE as an indicator of accuracy.

from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn import set_config
set_config(display="diagram")

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor


num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

col_trans = ColumnTransformer([
    ('num_trans', num_pipeline, num_cols),
    ('cat_trans', cat_pipeline, cat_cols),
])

model_pipeline = Pipeline([
    ('prep', col_trans),
    ('reg', LinearRegression())
])

model_pipeline

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num_trans',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['2020-2021_import',
                                                   '%Share_import',
                                                   '2020-2021_export',
                                                   '%Share_export',
                                                   '2020-2021_total',
                                                   '%Share_total']),
                                                 ('cat_trans',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Continent'])])),
                ('reg', LinearRegression())])

In [ ]:
def run_pipeline_cv(X_train, y_train, model=LinearRegression()):
    num_pipeline = Pipeline([
        ('scaler', StandardScaler())
    ])
    cat_pipeline = Pipeline([
        ('encoder', OneHotEncoder())
    ])

    col_trans = ColumnTransformer([
        ('num_trans', num_pipeline, num_cols),
        ('cat_trans', cat_pipeline, cat_cols),
    ])

    model_pipeline = Pipeline([
        ('prep', col_trans),
        ('reg', model)
    ])

    kf = KFold(n_splits=5, shuffle=True, random_state=123)

    model_cv = cross_val_score(model_pipeline, X_train, y_train, scoring='neg_mean_absolute_error', cv=kf)
    return model_cv

ms = []
es = []
st = []
for m in [LinearRegression(), KNeighborsRegressor(), RandomForestRegressor(), AdaBoostRegressor(), XGBRegressor()]:
    cur_model = m.__class__.__name__
    cur_score = run_pipeline_cv(X_train, y_train, model=m)
    cur_error = -1*cur_score.mean() # Since we used negative score in cross validation
    cur_error_std = cur_score.std()
    ms.append(cur_model)
    es.append(cur_error)
    st.append(cur_error_std)
test_result = pd.DataFrame({'Model':ms, 'MAE':es, 'MAE_std':st}).sort_values(by=['MAE'], ascending=True)

# visualization
fig = go.Figure()
fig.add_trace(go.Bar(x=test_result['Model'], y=test_result['MAE'], error_y=dict(type='data', array=test_result['MAE_std']), marker_color=INDIA_GREEN))
fig.update_layout(title_text=f'<b>Cross validation error by model</b>', template=TEMPLATE)
fig.update_yaxes(title='MAE')
fig.show()

# **Feature Selection**

In [ ]:
#One of the important things about linear regression is the validity of factor coefficients.
# I will configure the same process of the previous pipeline to check the pvalue for each feature.
#The pvalue for each feature in the results table below represents the level of the hypothesis test for whether the coefficient for each factor is zero.

# Split
test_df = df[df['Country'].isin(test_name)].reset_index(drop=True).copy()
train_df = df[~df['Country'].isin(test_name)].reset_index(drop=True).copy()

# Standard Scaling
for c in num_cols:
    ss = StandardScaler()
    train_df.loc[:, c] = ss.fit_transform(train_df[[c]])

# One-hot Encoding
for c in cat_cols:
    oh = OneHotEncoder(sparse=False)
    temp_ = pd.DataFrame(oh.fit_transform(train_df[[c]]))
    temp_.columns = [i.replace(" ", "")for i in oh.get_feature_names_out().tolist()]
    train_df = pd.concat([train_df.drop(columns=[c]), temp_], axis=1)

X_train = train_df.drop(columns=['Country', y_col])
y_train = train_df[y_col]

# Linear Regression
import statsmodels.api as sm # for summary
X_train_cons = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_cons).fit()
print(model.summary())


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



                            OLS Regression Results                            
Dep. Variable:        2021-2022_total   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     1378.
Date:                Mon, 10 Apr 2023   Prob (F-statistic):          7.18e-140
Time:                        16:04:27   Log-Likelihood:                -1379.3
No. Observations:                 158   AIC:                             2781.
Df Residuals:                     147   BIC:                             2814.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   4331

In [ ]:
# most of the factors are not valid, so I will recreate the model by removing the factors with high pvalue one by one.
# Split
test_df = df[df['Country'].isin(test_name)].reset_index(drop=True).copy()
train_df = df[~df['Country'].isin(test_name)].reset_index(drop=True).copy()

# Standard Scaling
for c in num_cols:
    ss = StandardScaler()
    train_df.loc[:, c] = ss.fit_transform(train_df[[c]])

# One-hot Encoding
for c in cat_cols:
    oh = OneHotEncoder(sparse=False)
    temp_ = pd.DataFrame(oh.fit_transform(train_df[[c]]))
    temp_.columns = [i.replace(" ", "")for i in oh.get_feature_names_out().tolist()]
    train_df = pd.concat([train_df.drop(columns=[c]), temp_], axis=1)

X_train = train_df.drop(columns=['Country', y_col])
test_cols = [
             #'2020-2021_import', # 1
             '%Share_import',
             #'2020-2021_export', # 2
             '%Share_export',
             #'2020-2021_total', # 4
             #'%Share_total', # 3
             # 'Continent_Africa', # 9
             # 'Continent_Asia', # 10
             #'Continent_Europe', # 6
             #'Continent_NorthAmerica', # 7
             # 'Continent_Oceania', # 11
             #'Continent_Others', # 5
             #'Continent_SouthAmerica' # 8
            ]
X_train = X_train[test_cols]
y_train = train_df[y_col]

# Linear Regression
import statsmodels.api as sm # for summary
X_train_cons = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_cons).fit()
print(model.summary())

# Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=123)
model_cv = cross_val_score(LinearRegression(), X_train, y_train, scoring='neg_mean_absolute_error', cv=kf)

# Append result
test_result = test_result.append(pd.DataFrame({'Model':['LinearRegression_FS'], 'MAE':[-1*model_cv.mean()], 'MAE_std':[model_cv.std()]})).sort_values(by=['MAE'], ascending=True)

# visualization
fig = go.Figure()
fig.add_trace(go.Bar(x=test_result['Model'], y=test_result['MAE'], error_y=dict(type='data', array=test_result['MAE_std']), marker_color=INDIA_GREEN))
fig.update_layout(title_text=f'<b>Cross validation error by model</b>', template=TEMPLATE)
fig.update_yaxes(title='MAE')
fig.show()

                            OLS Regression Results                            
Dep. Variable:        2021-2022_total   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     7172.
Date:                Mon, 10 Apr 2023   Prob (F-statistic):          1.76e-153
Time:                        16:09:36   Log-Likelihood:                -1380.3
No. Observations:                 158   AIC:                             2767.
Df Residuals:                     155   BIC:                             2776.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          4932.2960    120.955     40.778

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.

<ipython-input-35-2c344820ccb6>:47: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



# **Test**

In [ ]:
#Based on the final model, I made a model with the entire learning data and tested it. I used the value of 2020-2021 as the baseline.
#It can be seen that the MAE of the model is almost half lower than the baseline.


# Split
test_df = df[df['Country'].isin(test_name)].reset_index(drop=True).copy()
train_df = df[~df['Country'].isin(test_name)].reset_index(drop=True).copy()

# Standard Scaling
for c in num_cols:
    ss = StandardScaler()
    train_df.loc[:, c] = ss.fit_transform(train_df[[c]])
    test_df.loc[:, c] = ss.transform(test_df[[c]]) # test

# One-hot Encoding : No valid

# X & y
X_train = train_df.drop(columns=['Country', y_col])
y_train = train_df[y_col]
X_test = test_df.drop(columns=['Country', y_col])
y_test = test_df[['Country', y_col]]

# Linear Regression Feature Selection : Train & Test
fs = ['%Share_import', '%Share_export']
model = sm.OLS(y_train, sm.add_constant(X_train[fs])).fit()
y_pred = model.predict(sm.add_constant(X_test[fs]))

# Result
pred_df = y_test.copy()
pred_df['Pred'] = y_pred
## Baseline
pred_df = pd.merge(pred_df, df[['Country', '2020-2021_total']].rename(columns={'2020-2021_total':'Base'}), on='Country')

display(pred_df)

# visualization
fig = go.Figure()
fig.add_trace(go.Scatter(x=pred_df[y_col], y=pred_df['Pred'], text=pred_df['Country'], name='Pred',
                         mode = 'markers',
                         marker_color=INDIA_ORANGE)
             )
fig.add_trace(go.Scatter(x=pred_df[y_col], y=pred_df['Base'], text=pred_df['Country'], name='Base',
                         mode = 'markers',
                         marker_color=INDIA_GREEN)
             )

# Tune marker appearance and layout
fig.update_layout(title_text=f'<b>Pred vs Actual</b>', template=TEMPLATE)
fig.add_shape(type="line", x0=0, y0=0, x1=80000, y1=80000, line=dict(color="LightGray", width=5))
fig.update_yaxes(title="Pred")
fig.update_xaxes(title="Actual")
fig.show()

print(f"Model MAE : {mean_absolute_error(pred_df[y_col], pred_df['Pred'])}")
print(f"Baseline MAE : {mean_absolute_error(pred_df[y_col], pred_df['Base'])}")

,Country,2021-2022_total,Pred,Base
0,AFGHANISTAN,1065.40,2140.044612,1335.27
1,ANDORRA,0.05,176.454745,0.03
2,ARUBA,7.99,182.089322,3.60
3,BAHARAIN IS,1653.19,1717.798991,1075.74
4,BARBADOS,17.03,195.019821,11.71
...,...,...,...,...
62,US MINOR OUTLYING ISLANDS,0.46,178.348426,1.26
63,VENEZUELA,423.78,1978.516078,1271.34
64,VIRGIN IS US,4.72,295.138256,75.06
65,YEMEN REPUBLC,1074.38,1449.617862,808.93


Model MAE : 806.3473294776055
Baseline MAE : 1504.3489552238805
